In [1]:
import gc
import os
import pathlib
import faiss
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch
from tqdm import tqdm
from typing import Dict, List, Tuple, NamedTuple, Callable, Any
import scml
from scml import pandasx as pdx
tim = scml.Timer()
tim.start()
os.environ["TOKENIZERS_PARALLELISM"] = "false"
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()

In [2]:
sentences = [
    "Today the majority of humans own and operate cell phones on a daily basis. In essay form, explain if drivers should or should not be able to use cell phones in any capacity while operating a vehicle.",
    "Write an explanatory essay to inform fellow citizens about the advantages of limiting car usage. Your essay must be based on ideas and information that can be found in the passage set. Manage your time carefully so that you can read the passages; plan your response; write your response; and revise and edit your response. Be sure to use evidence from multiple sources; and avoid overly relying on one source. Your response should be in the form of a multiparagraph essay. Write your essay in the space provided.",
    "Some schools require students to complete summer projects to assure they continue learning during their break. Should these summer projects be teacher-designed or student-designed? Take a position on this question. Support your response with reasons and specific examples.",
    "You have just read the article, 'A Cowboy Who Rode the Waves.' Luke's participation in the Seagoing Cowboys program allowed him to experience adventures and visit many unique places. Using information from the article, write an argument from Luke's point of view convincing others to participate in the Seagoing Cowboys program. Be sure to include: reasons to join the program; details from the article to support Luke's claims; an introduction, a body, and a conclusion to your essay.",
    "Your principal has decided that all students must participate in at least one extracurricular activity. For example, students could participate in sports, work on the yearbook, or serve on the student council. Do you agree or disagree with this decision? Use specific details and examples to convince others to support your position.",
    "In 'The Challenge of Exploring Venus,' the author suggests studying Venus is a worthy pursuit despite the dangers it presents. Using details from the article, write an essay evaluating how well the author supports this idea. Be sure to include: a claim that evaluates how well the author supports the idea that studying Venus is a worthy pursuit despite the dangers; an explanation of the evidence from the article that supports your claim; an introduction, a body, and a conclusion to your essay.",
    "In the article 'Making Mona Lisa Smile,' the author describes how a new technology called the Facial Action Coding System enables computers to identify human emotions. Using details from the article, write an essay arguing whether the use of this technology to read the emotional expressions of students in a classroom is valuable.",
    "You have read the article 'Unmasking the Face on Mars.' Imagine you are a scientist at NASA discussing the Face with someone who thinks it was created by aliens. Using information in the article, write an argumentative essay to convince someone that the Face is just a natural landform.Be sure to include: claims to support your argument that the Face is a natural landform; evidence from the article to support your claims; an introduction, a body, and a conclusion to your argumentative essay.",
    "Some of your friends perform community service. For example, some tutor elementary school children and others clean up litter. They think helping the community is very important. But other friends of yours think community service takes too much time away from what they need or want to do. Your principal is deciding whether to require all students to perform community service. Write a letter to your principal in which you take a position on whether students should be required to perform community service. Support your position with examples.",
    "Your principal is considering changing school policy so that students may not participate in sports or other activities unless they have at least a grade B average. Many students have a grade C average. She would like to hear the students' views on this possible policy change. Write a letter to your principal arguing for or against requiring at least a grade B average to participate in sports or other activities. Be sure to support your arguments with specific reasons.",
    "In the article 'Driverless Cars are Coming,' the author presents both positive and negative aspects of driverless cars. Using details from the article, create an argument for or against the development of these cars. Be sure to include: your position on driverless cars; appropriate details from the article that support your position; an introduction, a body, and a conclusion to your argumentative essay.",
    "Write a letter to your state senator in which you argue in favor of keeping the Electoral College or changing to election by popular vote for the president of the United States. Use the information from the texts in your essay. Manage your time carefully so that you can read the passages; plan your response; write your response; and revise and edit your response. Be sure to include a claim; address counterclaims; use evidence from multiple sources; and avoid overly relying on one source. Your response should be in the form of a multiparagraph essay. Write your response in the space provided.",
    "Your principal is reconsidering the school's cell phone policy. She is considering two possible policies: Policy 1: Allow students to bring phones to school and use them during lunch periods and other free times, as long as the phones are turned off during class time. Policy 2: Do not allow students to have phones at school at all. Write a letter to your principal convincing her which policy you believe is better. Support your position with specific reasons.",
    "Some schools offer distance learning as an option for students to attend classes from home by way of online or video conferencing. Do you think students would benefit from being able to attend classes from home? Take a position on this issue. Support your response with reasons and examples.",
    "When people ask for advice, they sometimes talk to more than one person. Explain why seeking multiple opinions can help someone make a better choice. Use specific details and examples in your response.",
]

In [3]:
df = pd.read_parquet("input/preprocess.parquet")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39120 entries, 0 to 39119
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   essay_id        39120 non-null  int32 
 1   generated       39120 non-null  int8  
 2   source          39120 non-null  object
 3   prompt          39120 non-null  object
 4   text            39120 non-null  object
 5   text_bsc        39120 non-null  object
 6   text_bow        39120 non-null  object
 7   text_bow_len    39120 non-null  int16 
 8   prompt_bsc      39120 non-null  object
 9   prompt_bow      39120 non-null  object
 10  prompt_bow_len  39120 non-null  int16 
dtypes: int16(2), int32(1), int8(1), object(7)
memory usage: 2.4+ MB


In [4]:
pdx.value_counts(df["source"])

,count,percent
source,,
persuade_corpus,25797,0.659433
llammistral7binstruct,2420,0.061861
llama2_chat,2418,0.061810
original_moth,2260,0.057771
chat_gpt_moth,2124,0.054294
train_essays,1378,0.035225
llama_70b_v1,1172,0.029959
falcon_180b_v1,1051,0.026866
radek_500,500,0.012781


In [5]:
white = ["persuade_corpus", "train_essays"]
df = df[(df["generated"]==0) & (df["source"].isin(white))]
total = len(df)
df = df.sample(10_000)
sentences += df["text_bow"].tolist()
print(f"Sampled {len(sentences):,} out of {total:,} sentences")

Sampled 10015 out of 27172 sentences


In [6]:
%%time
model = SentenceTransformer("huggingface/sentence-transformers/all-mpnet-base-v2", device=torch.device("mps"))
model.max_seq_length = 384
em = model.encode(sentences=sentences, batch_size=512, show_progress_bar=True, convert_to_numpy=True)
print(f"em={em.shape}")

Batches:   0%|          | 0/20 [00:00<?, ?it/s]

em=(10015, 768)
CPU times: user 8.1 s, sys: 2.45 s, total: 10.6 s
Wall time: 2min 29s


In [7]:
faiss.normalize_L2(em)

In [8]:
%%time
d = em.shape[1]
index = faiss.IndexFlatIP(d)  # InnerProduct
index.verbose = True
index.train(em)
index.add(em)

CPU times: user 1.72 ms, sys: 4.35 ms, total: 6.07 ms
Wall time: 6.04 ms


In [9]:
%%time
faiss.write_index(index, "output/white.index")

CPU times: user 386 µs, sys: 3.61 ms, total: 3.99 ms
Wall time: 4.3 ms


In [10]:
%%time
index = faiss.read_index("output/white.index")
print(f"ntotal={index.ntotal}, is_trained={index.is_trained}")

ntotal=10015, is_trained=True
CPU times: user 1 ms, sys: 7.5 ms, total: 8.5 ms
Wall time: 8.11 ms


In [11]:
# sanity check
k=1
distances, ids = index.search(em[:20], k)  
print(f"I={repr(ids)}\nD={repr(distances)}")

I=array([[ 0],
       [ 1],
       [ 2],
       [ 3],
       [ 4],
       [ 5],
       [ 6],
       [ 7],
       [ 8],
       [ 9],
       [10],
       [11],
       [12],
       [13],
       [14],
       [15],
       [16],
       [17],
       [18],
       [19]])
D=array([[1.0000002 ],
       [0.99999994],
       [1.0000006 ],
       [0.99999994],
       [1.0000002 ],
       [1.0000001 ],
       [1.0000004 ],
       [1.0000005 ],
       [1.0000002 ],
       [1.0000001 ],
       [1.        ],
       [1.0000007 ],
       [1.        ],
       [1.        ],
       [1.0000002 ],
       [1.0000001 ],
       [0.9999998 ],
       [1.0000002 ],
       [1.0000002 ],
       [1.0000001 ]], dtype=float32)


In [12]:
queries = ["phone", "mars", "venus", "school projects", "school sports", "school activities", "cowboy", "facial emotion", "planets", "community service",
          "cars", "automobile", "tesla", "school policy", "politician", "government", "activist", "asking for advice", "distance learning", "learning outside the classroom",
           "baking class", 
          ]
q = model.encode(sentences=queries, batch_size=512, show_progress_bar=True, convert_to_numpy=True)
print(f"q={q.shape}")
faiss.normalize_L2(q)
k=1
distances, ids = index.search(q, k)
distances = distances.flatten()
ids = ids.flatten()
scores = list(zip(distances, queries))
scores.sort()
print(scores)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

q=(21, 768)
[(0.28400007, 'baking class'), (0.32608566, 'politician'), (0.32897663, 'activist'), (0.3492005, 'government'), (0.37236774, 'school projects'), (0.4410541, 'tesla'), (0.46479127, 'cowboy'), (0.47929913, 'cars'), (0.49908605, 'automobile'), (0.51537734, 'planets'), (0.5243023, 'mars'), (0.54078025, 'phone'), (0.55978537, 'community service'), (0.5868071, 'school sports'), (0.61108714, 'learning outside the classroom'), (0.65356404, 'venus'), (0.6612169, 'distance learning'), (0.6799522, 'facial emotion'), (0.6852598, 'school activities'), (0.68800247, 'school policy'), (0.739099, 'asking for advice')]


In [13]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

Total time taken 0:02:30.061522
